In [ ]:
from geostat import GP, Mesh, NormalizingFeaturizer
import matplotlib.pyplot as pp
import numpy as np

# Overview

In this notebook we will show how `Mesh` is used to make prediction locations. We'll generate synthetic data, fit a `GP` to it, and make predictions using `Mesh`.

# Synthesizing data

We will synthesize data at 200 random locations drawn from a 2D unit normal distribution.

In [ ]:
locs = np.random.normal(size = [200, 2])

Declare the terms of the spatial trend:

In [ ]:
def trend_terms(x, y): return x

Create a featurizer that the Gaussian process class `GP` will use to convert locations into trend features:

In [ ]:
featurizer = NormalizingFeaturizer(trend_terms, locs)

Instantiate a `GP` and immediately call `generate` to generate synthetic observations.
  * `parameter0` holds the geostatistical parameters for the `GP`.
  * `alpha` parameterizes the normal distribution prior for trend coefficients. 

In [ ]:
obs = GP(featurizer = featurizer,
         covariance_func = 'squared-exp',
         parameters = dict(range=0.5, sill=1., nugget=0.25),
         hyperparameters = dict(alpha=0.2),
         verbose=True).generate(locs)

When the data is plotted, you can see an overall trend with some localized variations.

In [ ]:
vmin, vmax = obs.min(), obs.max()
c = pp.scatter(locs[:, 0], locs[:, 1], c=obs, vmin=vmin, vmax=vmax)
pp.colorbar(c)
pp.title('Synthetic data')
pp.show()

# Inferring parameters

Now we create a second `GP`. This time we pass in the data (`locs` and `obs`) and it will fit the geospatial parameters to the data. Here `parameters` holds initial geospatial parameters, which are different that those in the first `GP`, but after fitting they converge to something close.

In [ ]:
gp = GP(featurizer = featurizer,
        covariance_func = 'squared-exp',
        parameters = dict(range=1.0, sill=0.5, nugget=0.5),
        hyperparameters = dict(alpha=obs.ptp()**2, reg=0, train_iters=300),
        verbose=True).fit(locs, obs)

# Generating predictions

Create mesh using bounding box of data, and crop to convex hull.

In [ ]:
mesh = Mesh(locs, nx=100).convex_hull()

Call `GP` to get predictions at mesh locations:

In [ ]:
mean, var = gp.predict(locs, obs, mesh.locations())

Create a slice for prediction mean and plot:

In [ ]:
meshx, meshy, value = mesh.slice(mean)
c = pp.pcolormesh(meshx, meshy, value, vmin=vmin, vmax=vmax)
pp.colorbar(c)
pp.title('Prediction mean')
pp.show()

Do the same for prediction variance:

In [ ]:
meshx, meshy, value = mesh.slice(var)
c = pp.pcolormesh(meshx, meshy, value, cmap='gist_heat_r')
pp.colorbar(c)
pp.title('Prediction variance')
pp.show()